# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [11]:
import urllib
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import matplotlib.pyplot as plt
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.as_matrix()
test = df_test.as_matrix()

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$. 
Try to understand each step of the learning algorithm and comment each line.

In [12]:
def learn_reg_ERM(X,y,lbda):
    # its number of epochs
    max_iter = 200
    # error
    e  = 0.001
    
    alpha = 1.
    # intial weight
    w = np.random.randn(X.shape[1]);
    for k in np.arange(max_iter):
        # y predicted 
        h = np.dot(X,w)
        # loss and its gradient 
        l,lg = loss(h, y)
        print ('loss: {}'.format(np.mean(l)))
        # regualizer and its gradient
        r,rg = reg(w, lbda)
        # overall gradient of loss function
        g = np.dot(X.T,lg) + rg 
        if (k > 0):
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old))
        # upadting parameter
        w = w - alpha * g
        if (np.linalg.norm(alpha * g) < e):
            break
        g_old = g
    return w

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient. 
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss at the points $h_i$. The partial derivative of the hinge loss $h_i$ with respect to the $i$-th position of the weight vector $\textbf{w}$ is $g_{i} = −y x_{i}$ if $l_{i} > 0$, else $g_{i} = 0$).

In [13]:
def loss(h, y):

    ##################
    l=np.maximum(0,1-h*y)
    g=np.where(l>0,-y,0)
    ##################
    return l, g

h =np.array([1,-1,1,1,-1])
y =np.array([6,4,-3,1,-1])
loss(h,y)

(array([0, 5, 4, 0, 0]), array([ 0, -4,  3,  0,  0]))

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$. 


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [14]:
def reg(w, lbda):
    ##################
    #INSERT CODE HERE#
    r = (lbda/2) * np.matmul(w.transpose(),w)
    g= lbda * w
    ##################
    return r, g

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [15]:
def predict(w, X):
    ##################
    #INSERT CODE HERE#
    preds = np.matmul(X, w)
    preds = np.where(preds>0,1,-1)
    ##################
    return preds

### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [16]:
##################
#INSERT CODE HERE#
def changeLabels(y_train,y_test):
    y_train[y_train == 0] = -1
    y_test[y_test == 0] = -1
    return y_train, y_test

def accuracy(y_actual,y_predicted):
    sum =(y_actual==y_predicted).sum()
    acc = sum/y_actual.shape[0]
    return acc
y_train, y_test= changeLabels(y_train,y_test)
w=learn_reg_ERM(X_train,y_train,5)
y_test_predicted = predict(w,X_test)
y_train_predicted = predict(w,X_train)
test_acc = accuracy(y_test,y_test_predicted)
train_acc = accuracy(y_train,y_train_predicted)
print(train_acc)
print(test_acc)
##################

loss: 0.9663431098102191
loss: 4.368355778190785
loss: 0.8945410872978204
loss: 1.6771505083279095
loss: 1.0407650813456215
loss: 0.9373972489097412
loss: 0.721905664550855
loss: 0.7055699312116649
loss: 0.7197878971557966
loss: 0.6945828376389477
loss: 0.693573265860242
loss: 0.697206488334946
loss: 0.6945733934097525
loss: 0.8275000000000002
loss: 0.7419036432141543
loss: 0.706252352587921
loss: 0.6925035499995941
loss: 0.6927491081726854
loss: 0.6937315718727867
loss: 0.696217954205944
loss: 0.693112125712897
loss: 0.6928838831724017
loss: 0.6928364442809861
loss: 0.6924806044755549
loss: 0.6924766690268989
loss: 0.6926576770410711
loss: 0.6925569497039239
loss: 0.7150000000000005
loss: 0.7028597629419371
loss: 0.6939768141826239
loss: 0.6922979331867298
loss: 0.6941723695896995
loss: 0.6926431415828992
loss: 0.6924958038339405
loss: 0.6923950637254697
loss: 0.6923487515718026
loss: 0.6923005948121851
loss: 0.6923327990564394
loss: 0.6924393648332752
loss: 0.6923131929342794
loss: 0

#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [17]:
##################
#INSERT CODE HERE#
clf = RandomForestClassifier()
clf.fit(X_train,y_train)
print(clf.score(X_test,y_test))
##################

0.7700534759358288


In [18]:
from sklearn.tree import DecisionTreeClassifier

clf_tree = DecisionTreeClassifier(criterion='entropy', max_depth=1)
clf_tree.fit(X_train, y_train)
clf_tree.score(X_test,y_test)

0.6149732620320856